<a href="https://colab.research.google.com/github/habataku/Animex/blob/master/%E5%BF%97%E6%9C%9B%E6%A0%A1%E5%88%86%E9%9B%A2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 最初に実行

In [9]:
#@title クラス定義 最初に実行 {display-mode:"form"}
import requests
import json
import re
import datetime
import pprint
from IPython.display import display,Markdown

class IStudyPlusUser:
  def __init__(self,json):
    self.data=json

  def followee(self):
    json=requests.get("https://api.studyplus.jp/2/users?followee="+self.data["username"]+"&page=1&per_page=100&include_recent_record_seconds=t").json()
    followees=json["users"]
    if json["page"]<json["total_page"]:
      for i in range(2,json["total_page"]+1):
        followees += requests.get("https://api.studyplus.jp/2/users?followee="+self.data["username"]+"&page="+str(i)+"&per_page=100&include_recent_record_seconds=t").json()["users"]
    return followees
  
  def follower(self):
    json=requests.get("https://api.studyplus.jp/2/users?follower="+self.data["username"]+"&page=1&per_page=100&include_recent_record_seconds=t").json()
    followers=json["users"]
    if json["page"]<json["total_page"]:
      for i in range(2,json["total_page"]+1):
        followers += requests.get("https://api.studyplus.jp/2/users?follower="+self.data["username"]+"&page="+str(i)+"&per_page=100&include_recent_record_seconds=t").json()["users"]
    return followers

class IStudyPlus:
  def __init__(self):
    self.api_root="https://api.studyplus.jp/2"
    result=requests.get("https://www.studyplus.jp/")
    self.web_session=result.cookies.get("_studyplus-web_session")
    self.cookies={"_studyplus-web_session":self.web_session,"AMP_TOKEN":"%24NOT_FOUND"}
    self.csrf_token=re.search(r'<meta name="csrf-token" content="(.{88})"',result.text).group(1)
    self.headers = {
            'authority': 'www.studyplus.jp',
            'accept': 'application/json, text/html, */*',
            'accept-language': 'ja,en-US;q=0.9,en;q=0.8',
            'cache': 'no-cache',
            'content-type': 'application/json',
            'origin': 'https://www.studyplus.jp',
            'referer': 'https://www.studyplus.jp',
            'sec-ch-ua': '"Google Chrome";v="111", "Not(A:Brand";v="8", "Chromium";v="111"',
            'sec-ch-mobile': '?0',
            'sec-ch-ua-platform': '"Windows"',
            'sec-fetch-dest': 'empty',
            'sec-fetch-mode': 'cors',
            'sec-fetch-site': 'same-origin',
            'user-agent': 'Mozilla/5.0 (iPad; CPU OS 13_3 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) CriOS/87.0.4280.77 Mobile/15E148 Safari/604.1 Edg/112.0.0.0',
            'x-csrf-token': self.csrf_token
    }

  def login(self,username,password,remember_me=False):
    data={"username":username,"password":password,"remenber_me":remember_me}
    result=requests.post("https://www.studyplus.jp/api/auth",data=json.dumps(data),cookies=self.cookies,headers=self.headers)
    self.web_session=result.cookies.get("_studyplus-web_session")
    self.auth=result.cookies.get('auth')
    self.cookies={"_studyplus-web_session":self.web_session,"auth":self.auth,"AMP_TOKEN":"%24NOT_FOUND"}
    self.username=re.search('username%22%3A%22(.*)%22%2C%22',self.auth).group(1)
    self.access_token=re.search('access_token%22%3A%22(.*)%22%7D',self.auth).group(1)
    self.headers["Authorization"]="OAuth "+self.access_token
    return self

  def __del__(self):
    if self.username:
      requests.post('https://www.studyplus.jp/auth',data="_method=delete&authenticity_token="+self.csrf_token,headers={
              'authority': 'www.studyplus.jp',
              "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
              "accept-language": "ja,en-US;q=0.9,en;q=0.8",
              "cache-control": "max-age=0",
              "content-type": "application/x-www-form-urlencoded",
              "sec-ch-ua": "\"Google Chrome\";v=\"113\", \"Chromium\";v=\"113\", \"Not-A.Brand\";v=\"24\"",
              "sec-ch-ua-mobile": "?0",
              "sec-ch-ua-platform": "\"Windows\"",
              "sec-fetch-dest": "document",
              "sec-fetch-mode": "navigate",
              "sec-fetch-site": "same-origin",
              "sec-fetch-user": "?1",
              "upgrade-insecure-requests": "1",
      })

  def me(self):
    if not self.username:
      return
    result=requests.get(self.api_root+"/users/"+self.username,cookies=self.cookies,headers=self.headers)
    self.web_session=result.cookies.get("_studyplus-web_session")
    self.cookies={"_studyplus-web_session":self.web_session,"auth":self.auth,"AMP_TOKEN":"%24NOT_FOUND"}
    return IStudyPlusUser(result.json())

In [10]:
#@title ログイン {display-mode:"form"}
メールアドレス="habatakutori.fbi@gmail.com"#@param {type:"string"}
パスワード="ctuisipnNian0972"#@param {type:"string"}
sp=IStudyPlus().login(メールアドレス,パスワード)
me=sp.me()

# **自動いいね**

In [ ]:
# @title 自分のタイムラインに指定した個数いいね {display-mode:"form"}
種類="\u30D5\u30A9\u30ED\u30FC\u4E2D"#@param ["フォロー中","定期テスト 成績アップ"]
if 種類=="フォロー中":
  timeline_url="followee"
elif 種類=="定期テスト 成績アップ":
  timeline_url="study_goal/test"
json=requests.get("https://api.studyplus.jp/2/timeline_feeds/"+timeline_url,cookies=sp.cookies,headers=sp.headers).json()
個数_30個単位=30#@param {type:"number"}

for i in range(0,個数_30個単位//30):
  for record in json["feeds"]:
    try:
      record.pop("feed_type")
      requests.post("https://api.studyplus.jp/2/timeline_events/"+str(list(record.values())[0]["event_id"])+"/likes/like",cookies=sp.cookies,headers=sp.headers).json()
    except:
      continue
  json=requests.get("https://api.studyplus.jp/2/timeline_feeds/"+timeline_url+"?until="+json["next"],cookies=sp.cookies,headers=sp.headers).json()

In [ ]:
# @title 自分のタイムラインに指定した個数いいね {display-mode:"form"}
種類="\u5B9A\u671F\u30C6\u30B9\u30C8 \u6210\u7E3E\u30A2\u30C3\u30D7"#@param ["フォロー中","定期テスト 成績アップ"]
if 種類=="定期テスト 成績アップ":
  timeline_url="study_goal/test"
elif 種類=="フォロー中":
  timeline_url="followee"
elif 種類=="英検準1級":
  timeline_url="study_goal/english-eiken-jun1"
elif 種類=="英検2級":
  timeline_url="study_goal/english-eiken-2"
elif 種類=="その他趣味・教養":
  timeline_url="study_goal/hobby-etc"
elif 種類=="その他":
  timeline_url="study_goal/skill-etc"
elif 種類=="北海道大学":
  timeline_url="study_goal/college-5"
elif 種類=="筑波大学":
  timeline_url="study_goal/college-86"
elif 種類=="千葉大学":
  timeline_url="study_goal/college-147"
elif 種類=="名古屋大学":
  timeline_url="study_goal/college-428"
elif 種類=="大阪大学":
  timeline_url="study_goal/college-524"
elif 種類=="神戸大学":
  timeline_url="study_goal/college-578"
elif 種類=="広島大学":
  timeline_url="study_goal/college-653"
elif 種類=="九州大学":
  timeline_url="study_goal/college-699"
elif 種類=="東京大学":
  timeline_url="study_goal/college-180"
elif 種類=="京都大学":
  timeline_url="study_goal/college-491"
elif 種類=="慶應義塾大学":
  timeline_url="study_goal/college-207"
elif 種類=="早稲田大学":
  timeline_url="study_goal/college-304"
json=requests.get("https://api.studyplus.jp/2/timeline_feeds/"+timeline_url,cookies=sp.cookies,headers=sp.headers).json()
個数_30個単位=240#@param {type:"number"}

for i in range(0,個数_30個単位//30):
  for record in json["feeds"]:
    try:
      record.pop("feed_type")
      requests.post("https://api.studyplus.jp/2/timeline_events/"+str(list(record.values())[0]["event_id"])+"/likes/like",cookies=sp.cookies,headers=sp.headers).json()
    except:
      continue
  json=requests.get("https://api.studyplus.jp/2/timeline_feeds/"+timeline_url+"?until="+json["next"],cookies=sp.cookies,headers=sp.headers).json()

In [ ]:
#@title 達成目標idから指定した個数いいね {display-mode:"form"}
達成目標id="college-225"# @param {type:"string"}
json=requests.get("https://api.studyplus.jp/2/timeline_feeds/study_goal/"+達成目標id,cookies=sp.cookies,headers=sp.headers).json()
個数_30個単位=300#@param {type:"number"}

for i in range(0,個数_30個単位//30):
  for record in json["feeds"]:
    try:
      record.pop("feed_type")
      requests.post("https://api.studyplus.jp/2/timeline_events/"+str(list(record.values())[0]["event_id"])+"/likes/like",cookies=sp.cookies,headers=sp.headers).json()
    except:
      continue
  json=requests.get("https://api.studyplus.jp/2/timeline_feeds/study_goal/"+達成目標id+"?until="+json["next"],cookies=sp.cookies,headers=sp.headers).json()

In [1]:
#@title 自分の達成目標id一覧取得 {display-mode:"form"}
md="|達成目標|id|\n|--|--|\n"
for val in me.data['study_goals']:
  md+=f"|{val['label']}|{val['key']}|\n"
display(Markdown(md))

NameError: ignored

In [ ]:
# @title 人を検索して全いいね
検索元="id\u6307\u5B9A"#@param ["フォロワー","フォロー中","id指定"]
名前またはid="b358c0cf40824c4ea677ac1cf5789986"#@param {type:"string"}

if 検索元=="フォロワー":
  user=next(filter(lambda x:x["nickname"]==名前またはid or x["username"]==名前またはid,me.follower()),None)
  json=requests.get("https://api.studyplus.jp/2/timeline_feeds/user/"+user["username"],cookies=sp.cookies,headers=sp.headers).json()
elif 検索元=="フォロー中":
  user=next(filter(lambda x:x["nickname"]==名前またはid or x["username"]==名前またはid,me.followee()),None)
  json=requests.get("https://api.studyplus.jp/2/timeline_feeds/user/"+user["username"],cookies=sp.cookies,headers=sp.headers).json()
else:
  json=requests.get("https://api.studyplus.jp/2/timeline_feeds/user/"+名前またはid,cookies=sp.cookies,headers=sp.headers).json()

for i in range(0,2000):
  count=0
  for record in json["feeds"]:
    try:
      record.pop("feed_type")
      body=list(record.values())[0]
      if body["if_you_like"]:
        count+=1
        continue
      requests.post("https://api.studyplus.jp/2/timeline_events/"+str(body["event_id"])+"/likes/like",cookies=sp.cookies,headers=sp.headers).json()
    except:
      continue
  if count==len(json["feeds"]) and i>100:
    break
  json=requests.get("https://api.studyplus.jp/2/timeline_feeds/user/"+user["username"]+"?until="+json["next"],cookies=sp.cookies,headers=sp.headers).json()

# **ユーザー情報**

In [ ]:
#@title 自分のフォロワー/フォロー中一覧 {display-mode:"form"}
モード="\u30D5\u30A9\u30ED\u30FC\u4E2D"#@param ["フォロワー","フォロー中"]
if モード=="フォロワー":
  userlist=me.follower()
elif モード=="フォロー中":
  userlist=me.followee()

md="|名前|id|\n|--|--|\n"
for val in userlist:
  md+=f"|{val['nickname']}|{val['username']}|\n"
display(Markdown(md))

|名前|id|
|--|--|
|じょ|eedc8f0a638f4cd2ab2bf6871ec84ba4|
|みーちゃん #雰囲気チェンジ？🤔|8dd5866988414ab29d60ecc3c8b100b5|
|なめくじ|37252caf61bc4c1282dc210fa394f27f|
|N|5ee35a66d7a744c6abeccdcfbaf24854|
|そたろ|937ad002d842496e91965cfb331bc387|
|夏の模試で九大B未満なら顔出す(絶対に嫌)|536547581a00492fa1238d989a89f644|
|ぬぇ💫|d8ea71ba3ac949d4ab082597a89404f0|
|たいやき|2a3e246484664a78a13bb7b8710dca59|
|まりんぬ|76a61a3d0bc44ea2a0300db05d47ba75|
|お奉行|72619865e6c74eb8b0cf4e032687e096|
|ちゅう|7a7755ca319b11e8b531064ca174cbd6|
|紫うさぎ|ab115b7b3ab64da9a0d248c1221c4546|
|♡코코나 ♡🐰🫧|48241de5feb04fc98352c03f374ccdc7|
|真央|65200e6ec6ac4c039ac718ee48d91ced|
|𝐰𝐚𝐤𝐚𝐧𝐚|42854e2481ae459982c97155817ee5b8|
|みゅいあす|f7686b48f381403ca8b9e649f0753187|
|👍🏻 ̖́-‬|bdf34117aac248fe8a573aca3f2ba9ab|
|い|de232e6ee465435c99aca786f7ffa3c8|
|澪|f2c9ed735b804c2f80f831dc74c41923|
|𝐑𝐢𝐧𝐧𝐨|1804224bd6c2443db35ad9942393babf|
|シャーク🦈|ca40b60e13994904bb664e436e4fc6e5|
|がき|771aba8be95e47159afe01f9c0496d50|
|しーばすお|da1c4d2df3a8455d8a18c8b46e579ea8|
|まきまきまっきー|bacd9a22163f4e02aae3a34948d447a2|
|くりおね🪼🩷|14744862661e4d979a0519308681d6a8|
|𝕐𝕦𝕜𝕚.∘ ❦ ໒꒱⋆⸜|157cc7d24a324c25a3a155431745b1a1|
|るるち🍊🌙|92d5d270573d46c8a9c84e127e01fc39|
|こいきっと|90c54d6b8cf648b6b98372fdf32f9839|
|雨夢|ad49c71a9507468b9d1c11491cf0e932|
|サメにゃん|a4b907806d954e7a8a708a4e2c935450|
|ひろの|511676e7308f4034975795771864643d|
|まとう🎤🍳|8f9d63c9c728465d9de8d272360e5dd2|
|みぃ❄️🌙中間6/6〜|0ad74e6a4117426e8b72f1da834c371e|
|きゅーこーみよし|2ca6749353e7485a93e9a98ae66b8d0c|
|かと。𓏸 𓈒 𓏸𓐍 🍵|58c21c72c8b74be4aac9b9cf0ff32fb6|
|たか|9d6f7190b15c4ed18beb4c557f309177|
|あいうえお|f05c89dbaaaa4840a4a35500c9804040|
|れいね|584f1b95ab8249f5a2e46c4e74f1773a|
|あづま ❃テスト期間中❃|6ded008c227944e0b23c402b4a01275f|
|佐久間(BRACKJACK)とうまとペア画中‼︎|8ee75b587fc1414d9797881b83f3db13|
|rk|34806f4b222e4c9c96d16596abc7c91a|
|怠惰な人間|326d6ab6f72e4663b49ab3db2036ab79|
|にぼし|66e939c45220433abd7b9b5ec477564f|
|ハズ|92de439b0c78436d884e6f8692203e0c|
|ぺんぎん|b358c0cf40824c4ea677ac1cf5789986|
|いつやるの　今でしょ|bca9ad31e5b5404d8f301be1a3987ff7|
|𝕄𝕚𝕫𝕦𝕜𝕚 #ていふ🫰🏻|2743991c3ddd487ba8a6db153b959a1a|
|ぼむ|dab44dc998884877af8250c70fc51380|
|ふめい|bd67cb635f014bddbf8f0298f90d2747|
|わさび|1d3e48ca85ae42f3ab42a9c175ee1372|
|‪‪❤︎‬|961088f4414c4bcc90802b301df8665e|
|あさやん|624b71f2f6764edaa5d7989401e07b78|
|し🍩|d1ec9178a55f4854b7fdb0bd617f1cbc|
|うさぎ|4658947aa8404e15a6a78be66d3d9651|
|内職王に俺はなる‼︎|4b9fe71e73474b74be45dbea045fd04d|
|Mucha|227cc5c9657e43e0b7d049058e1b846b|
|ao|892fcf71aae64e61b5477bcaa5cbc5d1|
|ぽむぽむ|c7bc44efa303436c91ef2a105977e0a1|
|カズ＠アルタイル　|a267a6a3af7d41539aa07e61c3bb579d|
|りん|61e7c1473c414e4a985bae23e4b8f826|
|みかん|26638132b66b42c79d095f40f56b91b2|
|まんｼﾞｭｳこわい|53cc0f3b6a2644c091ff3c90fe4691db|
|すけ|0993ac0b87d7438ab2d78193d5aa1abf|
|∫もっちー𝒹𝓉|affc4a31782f4d4cafacb9a0ee0b6aac|
|ほのか|2eaf295a41ff47b680ecb831760f3e3c|
|たぴおか|c0d4eee2dd3d47a99d8aece987b4ecfc|
|づづづ|b9d54d763687439b919cf4f9ccbd8003|
|カツタラ|daa0e16028704bf79252cb3fe253ccc9|
|ﾅﾅﾐ|af40fc060bab405db3ffae914c77219b|
|Asu|94d26fc568bc47c48a5fb2fd998205f0|
|みゆ|fcccbee16ae1420f9b3ea626e626cd74|
|七瀬‪𓂃 𓈒𓏸໒꒱グル画|2425ea0c8acc4a4787042757c7d99d6d|
|yuuya|cbd038d5184a4cb88082186a95e0ca41|
|🐣 𝙿𝙸𝚈𝙾 🍳|3973b90c924a42eaa06d25953daf45e0|
|明菜|883cbd4653e6437286b4f9f1f9212935|
|セキエイ|7872b557dd58470bb09ba32160017563|
|くろみつ|ebdea8f11a7b4a93b4b5683aafce7191|
|あー|c4e57e9e8b474a9bbfc92ee30905dd94|
|shallow|15e6adc6a3d541028a2c4f69e46a1c51|
|ゆっきー|8e1942df25e24671bc5a3a552e08e9ef|
|たけみち|fa9c87616d404168ae8d294613d298bf|
|わさび|468f464708f34978b7fe37a1ead64ddc|
|なぁ|4d3b533b285442898a47a84027dd9227|
|松ちゃん|a930b4d77fd440588c67d88df7fa5f60|
|ピクリン酸|686e41d1987d46fab4106a93f3dde4a2|
|ゆせい|b4b8bc509169423b875e98b5aa1f1c10|
|あいり|84b95892ee8743f8a769f876204ca4e2|
|もも様最高|cb10d8bc05884673951ad79d3a066bbd|
|ましゅ|5b3ba5a6fbf4499dabeea2c1b505eceb|
|無個性|8d284698e3dc434498cb2caebe3c6592|
|チロル|ba8c214db1e24722843ca0a3d7d457ca|
|Inferior|c43d6ba6dad747e0aeb15f5f748041d6|
|𝙎𝙖𝙠𝙪|a8cc63a1d6d14082933bb7b01d5065d4|
|🫧𝙮𝙪𝙞𝙠𝙖🫧|5af7171e70f045dcad3923309202348b|
|くるみリス|8a52010e411c457ea73549802e5fc974|
|ねおん|d6195448f8134c7f927972f594aaf38f|
|けいた|a6df52f69c7d4627b334d33cb702ad23|
|⋆̩jam︎︎☁︎︎*.|dc574957ae0f4f53a01cafb64b0cbdd8|
|𝑪𝒉𝒆𝒓𝒓𝒚|ba2a47d7847c46bcae42dce1d5c1d5ca|
|りょう@受験生|fc16e859ff71441e8fe08b55183bad0e|
|ジニア|815af1b3f37e4415bb0ca02dfc49ecc1|
|なのはな♪̊̈ ͙❁˚|e85c020bf86540f6a216bf4f358fe487|
|芹奈|9498996bdc2b4ffd9a45be18b733c78e|
|Nao♪|366086bbc11e49299be3035688e35d51|
|Mikiy|d600fe01af074794b0628adffe7d118a|
|きの|6ffde5791b164e0ebc188531c89b9a2b|
|iβu|23e48802753240afb1a55e7e8c0d5169|
|さんちゅう|b98be7f0ff604c998ac8f85d1be76c42|
|セーフリームニル|e7e75e1ac43f48ce8c1da965599be733|
|名無し＠studyplus非公式キャラクター|b9513ac08c98423088fc7b005c31a5e5|
|らぐ|11f804a720874e80a1b36e92e1f293c4|
|ななしさん|2c969cbcd5024341a602598d993383ea|
|ジョーイ|75ea555106674cc2a66d7a1365a5a416|
|𝙺|f9383d50c34249de982bca62915c6db9|
|キルア　凛🥀とペア画中🫥|2fd9530a35864852a8ca1a1b9434c6b1|
|G2|4c4e7f88171147ffa3b67cbd3ba935a7|
|りょうと|8bb1394639704e3dabb2fda782180075|
|ｻｿｻｿ|dbd3f319a97e4a368157462eec75aa09|
|kkk kkk|c1726a0e32bc439dbeedac66c75fafcc|
|ガラガラ|df9200489b1348709a01bd5ee8956813|
|とく|5bb03db7bd9a452b9ef9b070eb73ac60|
|ぷりん　@超低浮許してね|d45e9783acc641f8835b0aa0894d303f|
|yooooo|01418449b7924f51b1579c892908c668|
|しう|1fc3d2b220ce405cacb8d0db678f3aa1|
|ふじりょう|d122cc233217436999426ec20deb3cd2|
|FOREST|c80e5b13d6bc49d8b79f26ef8af8be58|
|こだまだこ|d3caee6adc584883965c16af7ebae4ed|
|誠|b295135a0e04469b9ef20398ef4cce6f|


In [ ]:
#@title 知り合いの知り合い一覧 {display-mode:"form"}
取得する人="\u30D5\u30A9\u30ED\u30EF\u30FC"#@param ["フォロワー","フォロー中"]
ユーザー名="1234567890"#@param {type:"string"}
モード="\u30D5\u30A9\u30ED\u30FC\u4E2D"#@param ["フォロワー","フォロー中"]

if 取得する人=="フォロワー":
  user=next(filter(lambda x:x["nickname"]==ユーザー名,me.follower()),None)
elif 取得する人=="フォロー中":
  user=next(filter(lambda x:x["nickname"]==ユーザー名,me.followee()),None)
print(f'{取得する人}の{ユーザー名} さんの{モード}一覧:')
if モード=="フォロワー":
  userlist=IStudyPlusUser(user).follower()
elif モード=="フォロー中":
  userlist=IStudyPlusUser(user).followee()

md="|名前|id|\n|--|--|\n"
for val in userlist:
  md+=f"|{val['nickname']}|{val['username']}|\n"
display(Markdown(md))

In [ ]:
#@title ユーザーidから情報を取得 {display-mode:"form"}
username="97a7afe10247462a99d6e858ede46674"#@param {type:"string"}
user=IStudyPlusUser(requests.get("https://api.studyplus.jp/2/users/"+username).json())
print(user.data["nickname"]+"さんの情報:")
pprint.pprint(user.data)

In [ ]:
# @title 没
# import requests
# from dataclasses import dataclass,fields
# from typing import Optional

# @dataclass
# class IStudyPlusUserProfile:
#   user_id: int
#   username: str
#   user_image_url: str
#   nickname: str
#   badge_type: str
#   user_organizations: list
#   disable_reply: bool
#   allow_send_image_message: bool
#   disable_profile_view: bool
#   goal: str
#   user_relationship_id: int
#   user_relationship_status: str
#   target_user_relationship_id: int
#   target_user_relationship_status: str
#   location_code: int
#   location: str
#   job_code: int
#   job: str
#   job_grade: int
#   follower_policy: str
#   feed_visibility: str
#   desired_departments: list
#   mute: bool
#   fav: bool
#   feed_is_visible: bool
#   study_goals: list

#   def followee(self):
#     json=requests.get("https://api.studyplus.jp/2/users?followee="+self.username+"&page=1&per_page=100&include_recent_record_seconds=t").json()
#     followees=json["users"]
#     if json["page"]<json["total_page"]:
#       for i in range(2,json["total_page"]+1):
#         followees |= requests.get(self.api_root+"/users?followee="+self.username+"&page="+i+"&per_page=100&include_recent_record_seconds=t").json()
#     followeeprofiles=[]
#     print(followees)
#     for follower in followees:
#       for field in fields(IStudyPlusRelateUser):
#         if field.name not in follower:
#           follower[field.name]=""
#       followeeprofiles.append(IStudyPlusRelateUser(**follower))
#     return followeeprofiles

# @dataclass
# class IStudyPlusUser(IStudyPlusUserProfile):
#   follow_count: int
#   follower_count: int
#   follow_requesting_count: int
#   biography: str
#   tags: list
#   educational_background: dict
#   material_count: int
#   premium: bool
#   book_user: bool
#   use_drill: bool
#   studyplus_confirmed: bool
#   studylog_confirmed: bool
#   record_count: int
#   record_hours_this_month: int
#   record_hours_last_month: int
#   total_record_hours: int
#   has_study_challenge: bool
#   sex: str
#   birthyear: int
#   birthmonth: int
#   birthdate: int
#   occupation: str
#   study_achievements: list
#   examination_count: int
#   email_is_confirmed: bool
#   learning_material_review_count: int
#   learning_material_review_like_count: int
#   desired_department_visibility: str
#   sex_visibility: str
#   birthyear_visibility: str
#   birthdate_visibility: str
#   location_visibility: str
#   job_visibility: str
#   occupation_visibility: str
#   study_goals_visibility: str
#   limited_message: bool

# @dataclass
# class IStudyPlusRelateUser(IStudyPlusUserProfile):
#   recent_record_seconds:int